In [2]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import IntegerParameter, ContinuousParameter, CategoricalParameter, HyperparameterTuner
from sagemaker import get_execution_role
import os

print("--- ⚙️ Step 1: Initializing SageMaker Environment ---")

# 1. 获取执行角色 (IAM Role)
# 如果在本地 VS Code 运行，可能需要手动指定 Role ARN
try:
    role = get_execution_role()
except ValueError:
    print("⚠️ 警告: 无法自动获取角色，请手动填入 ARN。")
    # 请去 AWS Console -> IAM -> Roles 找一个类似 AmazonSageMaker-ExecutionRole 的角色
    role = "arn:aws:iam::137568342316:role/SageMakerExecutionRole" 

# 2. 基础配置
sagemaker_session = sagemaker.Session()
bucket_name = 'sleep-disorder-mlops-bucket' # 您的 S3 桶名

# 3. 指定代码源 (Source of Truth for Code)
# SageMaker 会自动 `git clone` 这个仓库到训练实例中
git_repo = 'https://github.com/vinnie071015/sleeping-disorder-mlops.git' # 替换为您的仓库地址
git_config = {
    'repo': git_repo, 
    'branch': 'main'
}

print(f"✅ Role: {role}")
print(f"✅ Bucket: {bucket_name}")
print(f"✅ Git Repo: {git_repo}")


print("\n--- 💾 Step 2: Defining S3 Data Input ---")

# 定义 S3 数据输入
# SageMaker 会自动把这个 S3 路径下的所有文件下载到容器内的 /opt/ml/input/data/train/
# 这里的 s3_data 必须指向包含 sleep_data.csv 的文件夹路径 (以 / 结尾)
s3_input_path = f's3://{bucket_name}/raw_data/'

train_input = sagemaker.inputs.TrainingInput(
    s3_data=s3_input_path, 
    content_type='text/csv'
)

print(f"✅ Training Data Source: {s3_input_path}")


print("\n--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---")

# 使用 ml.m5.large (通用型)
instance_type = 'ml.m5.large'

# ⚠️ 关键修复：定义 S3 模型输出路径，强制使用你的自定义桶
model_output_s3_path = f's3://{bucket_name}/sagemaker-tuning-output/' 
print(f"✅ Model Artifacts Output Path (FIXED): {model_output_s3_path}")


common_estimator_args = {
    # 修正 1: 入口文件路径要包含 src (因为 source_dir 变成了根目录)
    'entry_point': 'src/train.py',
    
    # 修正 2: source_dir 设为 '.' (代表 Git 仓库的根目录)
    'source_dir': '.',
    
    'role': role,
    'instance_count': 1,
    'instance_type': instance_type,
    'framework_version': '1.2-1',
    'py_version': 'py3',
    'git_config': git_config,
    'sagemaker_session': sagemaker_session,
    
    # === 关键修复：添加 output_path 参数 ===
    'output_path': model_output_s3_path,
    'environment': {
        'WANDB_API_KEY': "0f759a15e3c54016f3f727c9720f0c9206fdd5c1",  # 容器会自动读取这个变量进行登录
        'WANDB_PROJECT': 'sleep-disorder-mlops', # 可选：指定项目名
        'WANDB_WATCH': 'false' # 可选：关闭不必要的模型监控以加快速度
    }
    # ========================================
}

print(f"✅ Instance Type: {instance_type}")
print(f"✅ Path Correction: source_dir='.', entry_point='src/train.py'")


print("\n--- 🎛️ Step 4: Defining Tuners for 3 Models ---")

# 定义指标抓取规则 (对应 src/train.py 中的 print 语句)
metric_definitions = [
    {'Name': 'accuracy', 'Regex': '✅ Accuracy: ([0-9\\.]+)'},
    {'Name': 'f1', 'Regex': '✅ F1 Score: ([0-9\\.]+)'}
]

# ==========================================
# A. Random Forest Tuner
# ==========================================
# 注意：这里使用的 common_estimator_args 已经包含了 output_path
rf_estimator = SKLearn(**common_estimator_args) 
# 固定模型类型为 RF
rf_estimator.set_hyperparameters(model_type='random_forest')

rf_tuner = HyperparameterTuner(
    estimator=rf_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'n_estimators': IntegerParameter(50, 150),
        'max_depth': IntegerParameter(5, 15)
    },
    max_jobs=2,          # 总共跑 2 次 (省钱)
    max_parallel_jobs=1, # 串行跑 (安全)
    base_tuning_job_name='rf-tuning'
)

# ==========================================
# B. SVM Tuner
# ==========================================
svm_estimator = SKLearn(**common_estimator_args)
svm_estimator.set_hyperparameters(model_type='svm')

svm_tuner = HyperparameterTuner(
    estimator=svm_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0),
        'kernel': CategoricalParameter(['rbf', 'linear'])
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='svm-tuning'
)

# ==========================================
# C. Logistic Regression Tuner
# ==========================================
lr_estimator = SKLearn(**common_estimator_args)
lr_estimator.set_hyperparameters(model_type='logistic_regression')

lr_tuner = HyperparameterTuner(
    estimator=lr_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0)
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='lr-tuning'
)

print("✅ Tuners for RF, SVM, and LR are ready.")

--- ⚙️ Step 1: Initializing SageMaker Environment ---


Couldn't call 'get_role' to get Role ARN from role name ML_Project to get Role path.
/Users/zhengchengsheng/opt/anaconda3/envs/mlops-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


⚠️ 警告: 无法自动获取角色，请手动填入 ARN。
✅ Role: arn:aws:iam::137568342316:role/SageMakerExecutionRole
✅ Bucket: sleep-disorder-mlops-bucket
✅ Git Repo: https://github.com/vinnie071015/sleeping-disorder-mlops.git

--- 💾 Step 2: Defining S3 Data Input ---
✅ Training Data Source: s3://sleep-disorder-mlops-bucket/raw_data/

--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---
✅ Model Artifacts Output Path (FIXED): s3://sleep-disorder-mlops-bucket/sagemaker-tuning-output/
✅ Instance Type: ml.m5.large
✅ Path Correction: source_dir='.', entry_point='src/train.py'

--- 🎛️ Step 4: Defining Tuners for 3 Models ---
✅ Tuners for RF, SVM, and LR are ready.


In [ ]:
import boto3
import time
from datetime import datetime
import sagemaker

# ==========================================
# 🛠️ 辅助函数定义 (已修复 bug)
# ==========================================

def get_tuning_job_status(tuner_obj):
    """获取 Tuner 当前 Job 的状态"""
    try:
        job_name = tuner_obj.latest_tuning_job.name
        response = boto3.client('sagemaker').describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=job_name)
        return response['HyperParameterTuningJobStatus']
    except Exception:
        return "Initializing"

def fetch_latest_error_log(bucket_name, prefix="debug_logs/"):
    """[失败时调用] 从 S3 获取最新生成的调试日志并打印"""
    print(f"\n🔍 [自动诊断] 检测到失败，正在从 s3://{bucket_name}/{prefix} 拉取最新日志...")
    s3 = boto3.client('s3')
    try:
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        if 'Contents' not in response:
            print("⚠️ 未在 S3 找到日志文件。")
            return
        files = sorted(response['Contents'], key=lambda x: x['LastModified'])
        latest_file = files[-1]
        key = latest_file['Key']
        print(f"📄 发现最新日志: {key} (时间: {latest_file['LastModified']})")
        print("="*60)
        file_obj = s3.get_object(Bucket=bucket_name, Key=key)
        print(file_obj['Body'].read().decode('utf-8')) 
        print("="*60)
    except Exception as e:
        print(f"❌ 拉取日志失败: {e}")

def print_best_model_info(tuner_obj, model_name):
    """
    [成功时调用] 获取并打印最佳模型的 S3 路径
    (修复了 'str' object has no attribute 'describe' 的问题)
    """
    try:
        # 1. 获取最佳 Job 的名称
        best_job_name = tuner_obj.best_training_job()
        
        if not best_job_name:
            print(f"⚠️ {model_name}: 任务显示完成，但未找到最佳 Training Job。")
            return

        # 2. 使用 boto3 直接查询 Job 详情 (这是最稳健的方法)
        sm_client = boto3.client('sagemaker')
        desc = sm_client.describe_training_job(TrainingJobName=best_job_name)
        
        # 3. 提取指标
        metrics = desc.get('FinalMetricDataList', [])
        # 找到 Accuracy 指标
        score = "N/A"
        for m in metrics:
            if m['MetricName'] == 'accuracy':
                score = m['Value']
                break
        
        # 4. 提取 S3 路径
        s3_uri = desc['ModelArtifacts']['S3ModelArtifacts']
        
        print(f"\n🏆 {model_name} 训练完成！")
        print(f"   ✅ 最佳准确率 (Accuracy): {score}")
        print(f"   💾 最佳模型保存位置: {s3_uri}")
        
    except Exception as e:
        print(f"⚠️ 获取 {model_name} 模型信息失败: {e}")

# ==========================================
# 🚀 主流程：并行启动
# ==========================================

print("\n--- 🚀 Step 5: Launching All Tuning Jobs in Parallel (Smart Monitor V2) ---")
print(f"🕒 开始时间: {datetime.now().strftime('%H:%M:%S')}\n")

# ⚠️ 如果你不想重新训练，请注释掉下面这三行 fit 代码
# ⚠️ 直接运行下面的监控循环，它会抓取刚才已经跑完的任务状态
rf_tuner.fit({'train': train_input}, wait=False) 
svm_tuner.fit({'train': train_input}, wait=False)
lr_tuner.fit({'train': train_input}, wait=False)

print("✅ 所有任务已在云端并行运行 (或已在运行中)！正在启动智能监控面板...\n")
print("-" * 90)
print(f"{'Time':<10} | {'Random Forest':<20} | {'SVM':<20} | {'Logistic Regression':<20}")
print("-" * 90)

# ==========================================
# 🕵️ 实时监控循环 (包含结果处理)
# ==========================================

processed_jobs = {'RF': False, 'SVM': False, 'LR': False}

try:
    while True:
        # 1. 获取最新状态
        rf_st = get_tuning_job_status(rf_tuner)
        svm_st = get_tuning_job_status(svm_tuner)
        lr_st = get_tuning_job_status(lr_tuner)
        
        current_time = datetime.now().strftime('%H:%M:%S')
        print(f"{current_time:<10} | {rf_st:<20} | {svm_st:<20} | {lr_st:<20}")
        
        # 2. 检查 RF 结果
        if rf_st in ['Completed', 'Failed', 'Stopped'] and not processed_jobs['RF']:
            if rf_st == 'Completed':
                print_best_model_info(rf_tuner, "Random Forest")
            elif rf_st == 'Failed':
                print(f"\n❌ Random Forest 任务失败！")
                fetch_latest_error_log(LOG_BUCKET_NAME) # 使用前面定义的桶名变量
            processed_jobs['RF'] = True 

        # 3. 检查 SVM 结果
        if svm_st in ['Completed', 'Failed', 'Stopped'] and not processed_jobs['SVM']:
            if svm_st == 'Completed':
                print_best_model_info(svm_tuner, "SVM")
            elif svm_st == 'Failed':
                print(f"\n❌ SVM 任务失败！")
                fetch_latest_error_log(LOG_BUCKET_NAME)
            processed_jobs['SVM'] = True

        # 4. 检查 LR 结果
        if lr_st in ['Completed', 'Failed', 'Stopped'] and not processed_jobs['LR']:
            if lr_st == 'Completed':
                print_best_model_info(lr_tuner, "Logistic Regression")
            elif lr_st == 'Failed':
                print(f"\n❌ Logistic Regression 任务失败！")
                fetch_latest_error_log(LOG_BUCKET_NAME)
            processed_jobs['LR'] = True

        # 5. 退出条件
        if all(processed_jobs.values()):
            print("-" * 90)
            print("\n🎉 所有调参任务已结束！")
            break
            
        time.sleep(30) 

except KeyboardInterrupt:
    print("\n⚠️ 监控已手动停止。")


--- 🚀 Step 5: Launching All Tuning Jobs in Parallel (Smart Monitor) ---
🕒 开始时间: 21:03:22

🌲 [1/3] Launching Random Forest... 

Cloning into '/var/folders/py/wsw9dv2n5g70rl8sm3pxxjg40000gn/T/tmpjkjodv70'...
Already on 'main'


Your branch is up to date with 'origin/main'.


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


✅ Job ID: rf-tuning-251203-2103
🛡️ [2/3] Launching SVM... 

...

Your branch is up to date with 'origin/main'.


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


✅ Job ID: lr-tuning-251203-2103

✨ 所有任务已在云端并行运行！正在启动智能监控面板...

------------------------------------------------------------------------------------------
Time       | Random Forest        | SVM                  | Logistic Regression 
------------------------------------------------------------------------------------------


/Users/zhengchengsheng/opt/anaconda3/envs/mlops-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


21:03:38   | InProgress           | InProgress           | InProgress          
21:04:13   | InProgress           | InProgress           | InProgress          
21:04:47   | InProgress           | InProgress           | InProgress          
21:05:23   | InProgress           | InProgress           | InProgress          
21:05:58   | InProgress           | InProgress           | InProgress          
21:06:33   | InProgress           | InProgress           | InProgress          
21:07:08   | InProgress           | InProgress           | InProgress          
21:07:42   | InProgress           | InProgress           | InProgress          
21:08:16   | InProgress           | InProgress           | InProgress          
21:08:51   | InProgress           | InProgress           | InProgress          
21:09:26   | InProgress           | Completed            | Completed           
⚠️ 获取 SVM 模型信息失败: 'str' object has no attribute 'describe'
⚠️ 获取 Logistic Regression 模型信息失败: 'str' object has no attribu

In [6]:
import boto3

def print_best_model_info_fixed(tuner_obj, model_name):
    try:
        # 1. 获取最佳 Job 的名称 (这是一个字符串)
        best_job_name = tuner_obj.best_training_job()
        
        if not best_job_name:
            print(f"⚠️ {model_name}: 未找到最佳 Training Job (可能是任务还没跑完或全失败了)。")
            return

        # 2. 使用 boto3 查询详情 (关键修复点)
        sm_client = boto3.client('sagemaker')
        desc = sm_client.describe_training_job(TrainingJobName=best_job_name)
        
        # 3. 提取准确率
        metrics = desc.get('FinalMetricDataList', [])
        score = "N/A"
        for m in metrics:
            if m['MetricName'] == 'accuracy': # 确保这里和你定义的 metric name 一致
                score = m['Value']
                break
        
        # 4. 提取 S3 模型路径
        s3_uri = desc['ModelArtifacts']['S3ModelArtifacts']
        
        print(f"\n🏆 {model_name} 结果报告")
        print(f"   🔹 最佳 Job 名称: {best_job_name}")
        print(f"   ✅ 最佳准确率: {score}")
        print(f"   💾 模型下载链接: {s3_uri}")
        
    except Exception as e:
        print(f"❌ 获取 {model_name} 信息失败: {e}")

# ==========================================
# 运行补救报告
# ==========================================
print("--- 📊 最终成绩单 (从已完成的任务中提取) ---")

# 只要你的 tuner 对象还在内存里，这段代码就能工作
print_best_model_info_fixed(rf_tuner, "Random Forest")
print_best_model_info_fixed(svm_tuner, "SVM")
print_best_model_info_fixed(lr_tuner, "Logistic Regression")

--- 📊 最终成绩单 (从已完成的任务中提取) ---

🏆 Random Forest 结果报告
   🔹 最佳 Job 名称: rf-tuning-251203-2103-001-5eb95c8f
   ✅ 最佳准确率: 0.8799999952316284
   💾 模型下载链接: s3://sleep-disorder-mlops-bucket/sagemaker-tuning-output/rf-tuning-251203-2103-001-5eb95c8f/output/model.tar.gz

🏆 SVM 结果报告
   🔹 最佳 Job 名称: svm-tuning-251203-2103-002-c8ccdacc
   ✅ 最佳准确率: 0.9067000150680542
   💾 模型下载链接: s3://sleep-disorder-mlops-bucket/sagemaker-tuning-output/svm-tuning-251203-2103-002-c8ccdacc/output/model.tar.gz

🏆 Logistic Regression 结果报告
   🔹 最佳 Job 名称: lr-tuning-251203-2103-002-81d8a1ae
   ✅ 最佳准确率: 0.9067000150680542
   💾 模型下载链接: s3://sleep-disorder-mlops-bucket/sagemaker-tuning-output/lr-tuning-251203-2103-002-81d8a1ae/output/model.tar.gz


In [1]:
import boto3
import sagemaker
import os
import joblib
import tarfile
import shutil

# 设置想要深入分析的冠军模型 (这里选 SVM，因为它准确率最高)
BEST_JOB_NAME = "svm-tuning-251203-2103-002-c8ccdacc" 
MODEL_S3_URI = "s3://sleep-disorder-mlops-bucket/sagemaker-tuning-output/svm-tuning-251203-2103-002-c8ccdacc/output/model.tar.gz"

def analyze_best_job(job_name):
    print(f"\n--- 🕵️‍♀️ 正在分析冠军任务: {job_name} ---")
    
    client = boto3.client('sagemaker')
    desc = client.describe_training_job(TrainingJobName=job_name)
    
    # 1. 提取超参数
    print("\n✅ 最佳超参数配置 (Secret Sauce):")
    hps = desc['HyperParameters']
    for k, v in hps.items():
        # 清理一下多余的引号，方便阅读
        clean_v = v.replace('"', '')
        print(f"   - {k}: {clean_v}")
        
    # 2. 提取所用环境
    image_uri = desc['AlgorithmSpecification']['TrainingImage']
    print(f"\n✅ 训练镜像: {image_uri}")
    
    return desc

def download_and_test_model(s3_uri):
    print(f"\n--- ⬇️ 正在下载模型进行本地验证 ---")
    
    local_tar = "best_model.tar.gz"
    extract_dir = "./best_model_extracted"
    
    # 清理旧文件
    if os.path.exists(extract_dir):
        shutil.rmtree(extract_dir)
    
    # 1. 下载
    sagemaker.s3.S3Downloader.download(s3_uri, ".")
    print(f"✅ 模型压缩包已下载: {local_tar}")
    
    # 重命名下载的文件（S3Downloader下载下来通常保持原名，但这里我们确保名字一致）
    # 注意：sagemaker download 会下载到当前目录，文件名就是 model.tar.gz
    # 如果当前目录已有 model.tar.gz，会被覆盖
    
    # 2. 解压
    with tarfile.open("model.tar.gz", "r:gz") as tar:
        tar.extractall(path=extract_dir)
        print(f"✅ 解压完成，目录内容: {os.listdir(extract_dir)}")
        
    # 3. 加载模型 (Sanity Check)
    try:
        model_path = os.path.join(extract_dir, "model.joblib")
        pipeline = joblib.load(model_path)
        print("\n🎉 模型加载成功！")
        print(f"   - Pipeline 结构: {pipeline}")
        
        # 打印一下 SVM 的具体参数，双重确认
        if 'classifier' in pipeline.named_steps:
            clf = pipeline.named_steps['classifier']
            print(f"   - 分类器参数: {clf.get_params()}")
            
    except Exception as e:
        print(f"❌ 模型加载失败: {e}")

if __name__ == "__main__":
    # 1. 分析参数
    analyze_best_job(BEST_JOB_NAME)
    
    # 2. 下载测试
    download_and_test_model(MODEL_S3_URI)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/zhengchengsheng/Library/Application Support/sagemaker/config.yaml

--- 🕵️‍♀️ 正在分析冠军任务: svm-tuning-251203-2103-002-c8ccdacc ---


/Users/zhengchengsheng/opt/anaconda3/envs/mlops-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)



✅ 最佳超参数配置 (Secret Sauce):
   - C: 4.37990419127643
   - _tuning_objective_metric: accuracy
   - kernel: linear
   - model_type: \svm\
   - sagemaker_container_log_level: 20
   - sagemaker_estimator_class_name: SKLearn
   - sagemaker_estimator_module: sagemaker.sklearn.estimator
   - sagemaker_job_name: sagemaker-scikit-learn-2025-12-03-13-03-28-764
   - sagemaker_program: src/train.py
   - sagemaker_region: us-east-1
   - sagemaker_submit_directory: s3://sleep-disorder-mlops-bucket/sagemaker-scikit-learn-2025-12-03-13-03-28-764/source/sourcedir.tar.gz

✅ 训练镜像: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3

--- ⬇️ 正在下载模型进行本地验证 ---
✅ 模型压缩包已下载: best_model.tar.gz
✅ 解压完成，目录内容: ['model.joblib', 'label_encoder.joblib']

🎉 模型加载成功！
   - Pipeline 结构: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['age', 'sleep_duration', 'quality_of_sleep', 'p